In [1]:
import pandas as pd
data_dir = "../data/"
output_dir = "../output/"

In [52]:
thresh = 0.00001 # 1e-5
species = pd.read_csv("../data/taxonomy.csv", index_col = 0).T
pathways = pd.read_csv("../data/pathways.csv", index_col = 0).T
# X = pd.concat([species, pathways], axis = 1)
X = species
X_old = X.copy()
X = X > thresh
X

Species,s__Abiotrophia_defectiva,s__Acidaminococcus_fermentans,s__Acidaminococcus_intestini,s__Actinomyces_graevenitzii,s__Actinomyces_odontolyticus,s__Actinomyces_oris,s__Actinomyces_turicensis,s__Actinomyces_viscosus,s__Adlercreutzia_equolifaciens,s__Aggregatibacter_segnis,...,s__Subdoligranulum_variabile,s__Succinatimonas_hippei,s__Sutterella_wadsworthensis,s__Turicibacter_sanguinis,s__Varibaculum_cambriense,s__Veillonella_atypica,s__Veillonella_dispar,s__Veillonella_parvula,s__Weissella_cibaria,s__Weissella_confusa
SAMD00036192,False,False,False,True,True,False,False,False,False,False,...,False,False,True,False,False,True,True,True,False,False
SAMD00036193,False,False,False,True,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
SAMD00036194,False,False,False,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
SAMD00036197,False,False,False,True,True,False,False,True,True,False,...,False,False,True,False,False,True,True,True,False,False
SAMD00036204,False,False,False,True,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN07534852,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
SAMN07534870,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
SAMN07534977,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
SAMN07534978,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False


In [53]:
y = pd.read_csv("../data/isHealthy.csv", index_col=0).T
y

,isHealthy
SAMD00036192,True
SAMD00036193,True
SAMD00036194,False
SAMD00036197,True
SAMD00036204,True
...,...
SAMN07534852,True
SAMN07534870,True
SAMN07534977,True
SAMN07534978,True


In [54]:
pivot = pd.concat([X[y['isHealthy']].mean(), X[~y['isHealthy']].mean()], axis = 1)
pivot.columns = ['healthy', 'not']
pivot['diff'] = pivot['healthy'] - pivot['not']
pivot['foldh'] = (pivot['healthy'] + 0.0001) / (pivot['not'] + 0.0001)
pivot['foldn'] = (pivot['not'] + 0.0001) / (pivot['healthy'] + 0.0001)

In [55]:
pivot['weight'] = 0
pivot

,healthy,not,diff,foldh,foldn,weight
Species,,,,,,
s__Abiotrophia_defectiva,0.006829,0.049679,-0.042850,0.139187,7.184578,0
s__Acidaminococcus_fermentans,0.040212,0.078317,-0.038104,0.514079,1.945225,0
s__Acidaminococcus_intestini,0.080046,0.115722,-0.035676,0.691973,1.445144,0
s__Actinomyces_graevenitzii,0.045144,0.122735,-0.077591,0.368332,2.714942,0
s__Actinomyces_odontolyticus,0.089909,0.155465,-0.065556,0.578595,1.728324,0
...,...,...,...,...,...,...
s__Veillonella_atypica,0.224962,0.344828,-0.119866,0.652491,1.532589,0
s__Veillonella_dispar,0.169575,0.264757,-0.095182,0.640628,1.560968,0
s__Veillonella_parvula,0.423369,0.583285,-0.159916,0.725883,1.377633,0


In [56]:
import numpy as np
pivot.loc[pivot['diff'] > 0, 'weight'] = np.log(pivot[pivot['diff'] > 0]['foldh'])
pivot

,healthy,not,diff,foldh,foldn,weight
Species,,,,,,
s__Abiotrophia_defectiva,0.006829,0.049679,-0.042850,0.139187,7.184578,0.0
s__Acidaminococcus_fermentans,0.040212,0.078317,-0.038104,0.514079,1.945225,0.0
s__Acidaminococcus_intestini,0.080046,0.115722,-0.035676,0.691973,1.445144,0.0
s__Actinomyces_graevenitzii,0.045144,0.122735,-0.077591,0.368332,2.714942,0.0
s__Actinomyces_odontolyticus,0.089909,0.155465,-0.065556,0.578595,1.728324,0.0
...,...,...,...,...,...,...
s__Veillonella_atypica,0.224962,0.344828,-0.119866,0.652491,1.532589,0.0
s__Veillonella_dispar,0.169575,0.264757,-0.095182,0.640628,1.560968,0.0
s__Veillonella_parvula,0.423369,0.583285,-0.159916,0.725883,1.377633,0.0


In [57]:
pivot.loc[pivot['diff'] < 0, 'weight'] = np.log(pivot[pivot['diff'] < 0]['foldn'])
pivot

,healthy,not,diff,foldh,foldn,weight
Species,,,,,,
s__Abiotrophia_defectiva,0.006829,0.049679,-0.042850,0.139187,7.184578,1.971937
s__Acidaminococcus_fermentans,0.040212,0.078317,-0.038104,0.514079,1.945225,0.665378
s__Acidaminococcus_intestini,0.080046,0.115722,-0.035676,0.691973,1.445144,0.368209
s__Actinomyces_graevenitzii,0.045144,0.122735,-0.077591,0.368332,2.714942,0.998770
s__Actinomyces_odontolyticus,0.089909,0.155465,-0.065556,0.578595,1.728324,0.547152
...,...,...,...,...,...,...
s__Veillonella_atypica,0.224962,0.344828,-0.119866,0.652491,1.532589,0.426958
s__Veillonella_dispar,0.169575,0.264757,-0.095182,0.640628,1.560968,0.445306
s__Veillonella_parvula,0.423369,0.583285,-0.159916,0.725883,1.377633,0.320367


In [58]:
pivot['weight2'] = abs(pivot['weight'] * pivot['diff'])
pivot

,healthy,not,diff,foldh,foldn,weight,weight2
Species,,,,,,,
s__Abiotrophia_defectiva,0.006829,0.049679,-0.042850,0.139187,7.184578,1.971937,0.084498
s__Acidaminococcus_fermentans,0.040212,0.078317,-0.038104,0.514079,1.945225,0.665378,0.025354
s__Acidaminococcus_intestini,0.080046,0.115722,-0.035676,0.691973,1.445144,0.368209,0.013136
s__Actinomyces_graevenitzii,0.045144,0.122735,-0.077591,0.368332,2.714942,0.998770,0.077496
s__Actinomyces_odontolyticus,0.089909,0.155465,-0.065556,0.578595,1.728324,0.547152,0.035869
...,...,...,...,...,...,...,...
s__Veillonella_atypica,0.224962,0.344828,-0.119866,0.652491,1.532589,0.426958,0.051178
s__Veillonella_dispar,0.169575,0.264757,-0.095182,0.640628,1.560968,0.445306,0.042385
s__Veillonella_parvula,0.423369,0.583285,-0.159916,0.725883,1.377633,0.320367,0.051232


In [59]:
X_good = X[pivot[pivot['diff'] > 0].nlargest(214,
    "weight2").index]
X_good_old = X_old[pivot[pivot['diff'] > 0].nlargest(214,
    "weight2").index]
X_good_old

Species,s__Bifidobacterium_angulatum,s__Bifidobacterium_catenulatum,s__Sutterella_wadsworthensis,s__Lachnospiraceae_bacterium_8_1_57FAA,s__Bifidobacterium_adolescentis,s__Bacteroidales_bacterium_ph8,s__Alistipes_senegalensis,s__Lactobacillus_ruminis,s__Barnesiella_intestinihominis,s__Lactobacillus_acidophilus,...,s__Prevotella_timonensis,s__Ruminococcus_obeum,s__Leuconostoc_lactis,s__Lachnospiraceae_bacterium_5_1_63FAA,s__Bacteroides_stercoris,s__Ruminococcus_sp_5_1_39BFAA,s__Bacteroides_xylanisolvens,s__Burkholderiales_bacterium_1_1_47,s__Parabacteroides_distasonis,s__Bacteroides_thetaiotaomicron
SAMD00036192,0.0,0.004416,0.006551,0.000000,0.022290,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000334,0.000000,0.001515,0.005369,0.013915,0.000046,0.000000,0.012953,0.000720
SAMD00036193,0.0,0.000228,0.000183,0.000000,0.227322,0.000103,0.000000,0.0,0.000419,0.0,...,0.0,0.020326,0.000000,0.006380,0.002153,0.000000,0.000000,0.000000,0.000497,0.000502
SAMD00036194,0.0,0.000000,0.001943,0.000000,0.090593,0.000212,0.000000,0.0,0.000000,0.0,...,0.0,0.004725,0.000000,0.005432,0.000000,0.071123,0.000210,0.000000,0.000000,0.000000
SAMD00036197,0.0,0.000000,0.007306,0.000000,0.091049,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.002038,0.000000,0.000138,0.000005,0.021714,0.000000,0.000000,0.001156,0.000003
SAMD00036204,0.0,0.000000,0.000000,0.000000,0.070562,0.000763,0.000000,0.0,0.001186,0.0,...,0.0,0.004646,0.000000,0.001777,0.000000,0.074059,0.000045,0.001589,0.043859,0.000852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN07534852,0.0,0.000000,0.007777,0.001208,0.000000,0.060104,0.000712,0.0,0.000000,0.0,...,0.0,0.002602,0.000000,0.003540,0.000924,0.000000,0.022522,0.000042,0.007260,0.018413
SAMN07534870,0.0,0.000245,0.005217,0.000000,0.013176,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.008954,0.000616,0.001483,0.123784,0.017542,0.000000,0.000000,0.014137,0.007061
SAMN07534977,0.0,0.000000,0.000000,0.000000,0.010072,0.007427,0.000000,0.0,0.000000,0.0,...,0.0,0.006962,0.000000,0.000000,0.000000,0.010363,0.032723,0.003143,0.000000,0.045579
SAMN07534978,0.0,0.001325,0.001308,0.000000,0.035911,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.018653,0.002762,0.003116,0.057697,0.032940,0.000000,0.000000,0.011983,0.003386


In [60]:
X_bad = X[pivot[pivot['diff'] < 0].nlargest(214,
    "weight2").index]
X_bad_old = X_old[pivot[pivot['diff'] < 0].nlargest(214,
    "weight2").index]
X_bad_old

Species,s__Solobacterium_moorei,s__Granulicatella_adiacens,s__Clostridium_bolteae,s__Peptostreptococcus_stomatis,s__Clostridium_ramosum,s__Clostridium_symbiosum,s__candidate_division_TM7_single_cell_isolate_TM7c,s__Clostridiales_bacterium_1_7_47FAA,s__Clostridium_hathewayi,s__Gemella_morbillorum,...,s__Atopobium_vaginae,s__Bacteroides_sp_4_3_47FAA,s__Megamonas_funiformis,s__Prevotella_disiens,s__Lactobacillus_sakei,s__Haemophilus_sputorum,s__Dialister_succinatiphilus,s__Succinatimonas_hippei,s__Clostridium_bartlettii,s__Ruminococcus_torques
SAMD00036192,0.000074,0.000056,0.000004,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017826
SAMD00036193,0.000000,0.000000,0.000331,0.0,0.000000,0.000000,0.0,0.000000,0.000222,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.001019
SAMD00036194,0.000000,0.000000,0.000651,0.0,0.000868,0.000100,0.0,0.000000,0.000157,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001578,0.013552
SAMD00036197,0.000144,0.000280,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000885,0.010093
SAMD00036204,0.000000,0.000000,0.000887,0.0,0.013896,0.000000,0.0,0.000089,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000287,0.005705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN07534852,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002558,0.004263
SAMN07534870,0.000000,0.000000,0.000000,0.0,0.000000,0.001462,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001559,0.016653
SAMN07534977,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.009821
SAMN07534978,0.000000,0.000000,0.000000,0.0,0.000000,0.001096,0.0,0.000000,0.000099,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001858,0.062468


In [61]:
theta_good = pivot[pivot['diff'] > 0]['weight2'].sort_values(
ascending=False)
print(theta_good.shape)
theta_good

(108,)


Species
s__Bifidobacterium_angulatum              2.133969e-01
s__Bifidobacterium_catenulatum            1.431894e-01
s__Sutterella_wadsworthensis              1.320367e-01
s__Lachnospiraceae_bacterium_8_1_57FAA    9.079485e-02
s__Bifidobacterium_adolescentis           8.446548e-02
                                              ...     
s__Ruminococcus_sp_5_1_39BFAA             9.972740e-05
s__Bacteroides_xylanisolvens              9.507766e-06
s__Burkholderiales_bacterium_1_1_47       3.810323e-06
s__Parabacteroides_distasonis             5.077062e-07
s__Bacteroides_thetaiotaomicron           4.777555e-08
Name: weight2, Length: 108, dtype: float64

In [62]:
theta_bad = pivot[pivot['diff'] < 0]['weight2'].nlargest(214)
print(theta_bad.shape)
theta_bad

(205,)


Species
s__Solobacterium_moorei           0.364616
s__Granulicatella_adiacens        0.332135
s__Clostridium_bolteae            0.248070
s__Peptostreptococcus_stomatis    0.235289
s__Clostridium_ramosum            0.228232
                                    ...   
s__Haemophilus_sputorum           0.000217
s__Dialister_succinatiphilus      0.000185
s__Succinatimonas_hippei          0.000181
s__Clostridium_bartlettii         0.000056
s__Ruminococcus_torques           0.000037
Name: weight2, Length: 205, dtype: float64

In [63]:
y = y.to_numpy()
y

array([[ True],
       [ True],
       [False],
       ...,
       [ True],
       [ True],
       [ True]])

In [64]:
y = y.flatten()
y

array([ True,  True, False, ...,  True,  True,  True])

In [65]:
healthyIdx = np.where(y)[0]
unhealthyIdx = np.where(~y)[0]
unhealthyIdx

array([   2,    6,    9, ..., 4337, 4338, 4341])

In [66]:
theta_good.sum(), theta_bad.sum()

(1.990208795371848, 12.376674530584733)

In [67]:
X_good2 = X_good.to_numpy().astype(float)
X_bad2 = X_bad.to_numpy().astype(float)
theta_good2 = theta_good.to_numpy()
theta_bad2 = theta_bad.to_numpy()

In [68]:
theta_goods = np.vstack([theta_good2] * theta_good2.shape[0])
for i in range(theta_good2.shape[0]):
    theta_goods[i, i:] = 0
theta_goods[10]

array([0.21339686, 0.14318936, 0.13203671, 0.09079485, 0.08446548,
       0.07688577, 0.07072288, 0.06929234, 0.05286422, 0.04958672,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [69]:
theta_bads = np.vstack([theta_bad2] * theta_bad2.shape[0])
for i in range(theta_bad2.shape[0]):
    theta_bads[i, i:] = 0
theta_bads[20, :]

array([0.36461562, 0.33213498, 0.24806993, 0.2352892 , 0.22823211,
       0.21845839, 0.21606379, 0.21561245, 0.21471895, 0.21385063,
       0.20287926, 0.19940627, 0.19744876, 0.194339  , 0.19308637,
       0.17838159, 0.1755825 , 0.17516987, 0.16130225, 0.15652798,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [70]:
# 125, 203
from numba import njit
@njit(nogil=True)
def getChi(X_good, X_bad,
           theta_goods, theta_bads, healthyIdx, unhealthyIdx, num_good, num_bad):
    good_term = (X_good @ theta_goods[num_good, :])
    bad_term = (X_bad @ theta_bads[num_bad, :])
    y_hat = good_term - bad_term
    y_hat = y_hat.reshape(len(y_hat), 1)
    p_h = (y_hat[healthyIdx] > 0).sum() / 2636
    p_n = (y_hat[unhealthyIdx] < 0).sum() / 1711
    return 0.5 * (p_h + p_n)

In [71]:
%%time
getChi(X_good2, X_bad2,
        theta_goods, theta_bads,
        healthyIdx, unhealthyIdx, 
      166, 29)

CPU times: user 546 ms, sys: 4.36 ms, total: 551 ms
Wall time: 545 ms


0.46201052016364696

In [74]:
# from numba import prange
# @njit(nogil=True, parallel=True)
def run():
    scores = [0.0]
    for num_good in range(theta_good2.shape[0]):
        for num_bad in range(theta_bad2.shape[0]):
            chi = getChi(X_good2, X_bad2, 
                theta_goods, theta_bads,
                healthyIdx, unhealthyIdx, 
                num_good, num_bad)
            scores.append(chi)
    return scores

In [75]:
%%time
scores = run()
max(scores)

CPU times: user 20.1 s, sys: 155 ms, total: 20.3 s
Wall time: 2.55 s


0.73083608783299

In [313]:
scores.index(max(scores)) % theta_bad2.shape[0]

29

In [82]:
pivot[pivot['diff'] > 0].nlargest(166, 'weight2')['weight2'].sum(), \
    pivot[pivot['diff'] < 0].nlargest(29, 'weight2')['weight2'].sum()

(2.326063118459373, 5.601553499423158)

In [88]:
X

,s__Abiotrophia_defectiva,s__Acidaminococcus_fermentans,s__Acidaminococcus_intestini,s__Actinomyces_graevenitzii,s__Actinomyces_odontolyticus,s__Actinomyces_oris,s__Actinomyces_turicensis,s__Actinomyces_viscosus,s__Adlercreutzia_equolifaciens,s__Aggregatibacter_segnis,...,TRNA-CHARGING-PWY: tRNA charging,TRPSYN-PWY: L-tryptophan biosynthesis,TYRFUMCAT-PWY: L-tyrosine degradation I,UBISYN-PWY: superpathway of ubiquinol-8 biosynthesis (prokaryotic),UDPNACETYLGALSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis II,UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,URDEGR-PWY: superpathway of allantoin degradation in plants,URSIN-PWY: ureide biosynthesis,VALDEG-PWY: L-valine degradation I,VALSYN-PWY: L-valine biosynthesis
SAMD00036192,False,False,False,True,True,False,False,False,False,False,...,True,True,False,True,False,True,False,False,False,True
SAMD00036193,False,False,False,True,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036194,False,False,False,False,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036197,False,False,False,True,True,False,False,True,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036204,False,False,False,True,True,True,False,False,True,False,...,True,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN07534852,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,True,True,False,False,True
SAMN07534870,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,True,True,False,False,True
SAMN07534977,False,False,False,False,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMN07534978,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,True,True,False,False,True


In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(C=0.02, fit_intercept=False, max_iter=500, random_state=42).fit(
    X_train, y_train)
clf.score(X_test, y_test)

0.7781609195402299

In [137]:
clf.score(X_test, y_test)

0.7781609195402299

In [157]:
pivot3 = pivot2.copy()
del pivot3['weight2']
del pivot3['weight3']
pivot3['coef'] = clf.coef_[0]
pivot3['adiff'] = abs(pivot3['diff'])
pivot3.loc[pivot3['diff'] < 0, 'weight'] *= -1
pivot3

,healthy,not,diff,foldh,foldn,weight,coef,adiff
s__Abiotrophia_defectiva,0.006829,0.049679,-0.042850,0.139187,7.184578,-1.971937,-0.021747,0.042850
s__Acidaminococcus_fermentans,0.040212,0.078317,-0.038104,0.514079,1.945225,-0.665378,-0.021747,0.038104
s__Acidaminococcus_intestini,0.080046,0.115722,-0.035676,0.691973,1.445144,-0.368209,-0.021747,0.035676
s__Actinomyces_graevenitzii,0.045144,0.122735,-0.077591,0.368332,2.714942,-0.998770,-0.021747,0.077591
s__Actinomyces_odontolyticus,0.089909,0.155465,-0.065556,0.578595,1.728324,-0.547152,-0.021747,0.065556
...,...,...,...,...,...,...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.991275,0.990064,0.001210,1.001222,0.998779,0.001222,-0.021747,0.001210
URDEGR-PWY: superpathway of allantoin degradation in plants,0.295903,0.443016,-0.147113,0.668003,1.496998,-0.403462,-0.021747,0.147113
URSIN-PWY: ureide biosynthesis,0.003414,0.004091,-0.000677,0.838491,1.192618,-0.176151,-0.021747,0.000677
VALDEG-PWY: L-valine degradation I,0.006449,0.003507,0.002942,1.815822,0.550715,0.596539,-0.021747,0.002942


In [160]:
X_w = pivot3[['weight', 'adiff']]
y_w = clf.coef_[0]
y_w

s__Abiotrophia_defectiva                                      -0.021747
s__Acidaminococcus_fermentans                                 -0.021747
s__Acidaminococcus_intestini                                  -0.021747
s__Actinomyces_graevenitzii                                   -0.021747
s__Actinomyces_odontolyticus                                  -0.021747
                                                                 ...   
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I      -0.021747
URDEGR-PWY: superpathway of allantoin degradation in plants   -0.021747
URSIN-PWY: ureide biosynthesis                                -0.021747
VALDEG-PWY: L-valine degradation I                            -0.021747
VALSYN-PWY: L-valine biosynthesis                             -0.021747
Name: coef, Length: 844, dtype: float64

In [161]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression(fit_intercept=False).fit(
    X_w, y_w)
clf.score(X_w, y_w)

-5.762893563688138e+30